In [40]:
import os
from pathlib import Path
from importlib import reload

import numpy as np
import torch
import torch.nn as nn

from torch.optim import Adam
from torch.optim import SGD
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch_optimizer import RAdam

from enot.models import SearchSpaceModel
from enot.models.mobilenet import build_mobilenet
from enot.optimize import EnotPretrainOptimizer
from enot.optimize import EnotSearchOptimizer

from enot_utils.metric_utils import accuracy
from enot_utils.schedulers import WarmupScheduler

<module 'data' from 'C:\\Users\\daddu\\Desktop\\css\\practice_1\\data.py'>

In [10]:
PATH = Path().absolute()
ENOT_HOME_DIR = PATH / "enot"
ENOT_DATASETS_DIR = PATH / "datasets"
PROJECT_DIR = PATH / "search"

ENOT_HOME_DIR.mkdir(exist_ok=True)
ENOT_DATASETS_DIR.mkdir(exist_ok=True)
PROJECT_DIR.mkdir(exist_ok=True)

### data

datasets:
1. train
2. validation
3. search
4. test
5. tune

loaders:
1. pretrain: train, val
2. seach: train, val
3. tune: train, val (same as pretrain)

In [17]:
dataloaders = data.get_loaders(1)

In [18]:
dataloaders

{'pretrain': {'train': None, 'val': None},
 'search': {'train': None, 'val': None},
 'tune': {'train': None, 'val': None}}

### search space

In [4]:
SEARCH_OPS = [
    "MIB_k=3_t=6",
    "MIB_k=5_t=6",
    "MIB_k=7_t=6",
]

model = build_mobilenet(
    search_ops = SEARCH_OPS,
    num_classes = 10,
    blocks_out_channels= [24, 32, 64, 96, 160, 320],
    blocks_count = [2, 2, 2, 1, 2, 1],
    blocks_stride = [2, 2, 2, 1, 2, 1],
)

search_space = SearchSpaceModel(model).cuda()

### Pretrain

In [6]:
def train(search_space, 
          train_loader, valid_loader, 
          optimizer, enot_optimizer, 
          n_epochs, n_warmup_epochs, phase="pretrain",
          latency_loss_weight = 0):
    
    len_train_loader = len(train_loader)
    scheduler = None
    if phase != "tune":
        scheduler = CosineAnnealingLR(optimizer, T_max = len_train_loader * n_epochs, eta_min = 1e-8)
    if n_warmup_epochs > 0:
        scheduler = WarmupScheduler(scheduler, warmup_steps = len_train_loader * n_warmup_epochs)

    metric_function = accuracy
    loss_function = nn.CrossEntropyLoss().cuda()

    for epoch in range(n_epochs):
        print(f"Epoch #{epoch}")

        search_space.train()
        train_metrics = {
            "loss": 0.0,
            "accuracy": 0.0,
            "n": 0,
        }

        for inputs, labels in train_loader:
            if phase == "pretrain":
                if not search_space.initialize_output_distribution_enabled:
                    search_space.initialize_output_distribution_optimization(inputs)
            if phase == "tune":
                optimizer.zero_grad()
            else:
                enot_optimizer.zero_grad()

            def closure():
                pred = search_space(inputs)
                loss = loss_function(pred, labels)
                
                if phase == "search" and latency_loss_weight > 0:
                    loss += search_space.loss_latency_expectation * latency_loss_weight
                    
                loss.backward()
                metric = metric_function(pred, labels)

                train_metrics["loss"] += loss.item()
                train_metrics["accuracy"] += metric.item()
                train_metrics["n"] += 1
            
            if phase == "tune":
                closure()
                optimizer.step()
            else:
                enot_optimizer.step(closure)
            
            if scheduler is not None:
                scheduler.step()

        train_loss = train_metrics["loss"] / train_metrics["n"]
        train_accuracy = train_metrics["accuracy"] / train_metrics["n"]
        print("train metrics:")
        print(f"\tloss: {train_loss}")
        print(f"\taccuracy: {train_accuracy}")
        if phase == "search":
            arch_probabilities = np.array(search_space.architecture_probabilities)
            print(f"\tarch_probabilities: {arch_probabilities}")

        search_space.eval()
        if phase == "search":
            search_space.sample_best_arch()
        
        valid_loss = 0
        valid_accuracy = 0
        for inputs, labels in valid_loader:
            if phase == "pretrain":
                search_space.sample_random_arch()

            pred = search_space(inputs)
            loss = loss_function(pred, labels)
            metric = metric_function(pred, labels)

            validation_loss += batch_loss.item()
            validation_accuracy += batch_metric.item()

        n = len(validation_loader)
        valid_loss /= n
        valid_accuracy /= n

        print("validation metrics:")
        print(f"\tloss: {valid_loss}")
        print(f"\taccuracy: {valid_accuracy}")
        if phase == "search" and search_space.latency_type is not None:
            latency = search_space.forward_latency.item()
            print(f"\tlatency: {latency}")
        print()


In [ ]:
N_EPOCHS = 3
N_WARMUP_EPOCHS = 1
LR = 6e-2

train_loader = dataloaders["pretrain"]["train"]
valid_loader = dataloaders["pretrain"]["val"]

optimizer = SGD(params = search_space.model_parameters(), 
                lr=LR, momentum = 0.9, weight_decay = 1e-4)
enot_optimizer = EnotPretrainOptimizer(search_space = search_space, optimizer = optimizer)

train(search_space = search_space,
      train_loader = train_loader,
      valid_loader = valid_loader,
      optimizer = optimizer,
      enot_optimizer = enot_optimizer,
      n_epochs = N_EPOCHS,
      n_warmup_epochs = N_WARMUP_EPOCHS,
      phase = "pretrain")

### Search

In [7]:
N_EPOCHS = 10
latency_loss_weight = 2e-3

optimizer = RAdam(search_space.architecture_parameters(), lr=0.01)
enot_optimizer = EnotSearchOptimizer(search_space, optimizer, latency_type='mmac')

train_loader = dataloaders['search']["train"]
valid_loader = dataloaders['search']["val"]

train(search_space = search_space,
      train_loader = train_loader,
      valid_loader = valid_loader,
      optimizer = optimizer,
      enot_optimizer = enot_optimizer,
      n_epochs = N_EPOCHS,
      phase = "search",
      latency_loss_weight = latency_loss_weight)

### Tune

In [8]:
best_model = search_space.get_network_with_best_arch().cuda()

N_EPOCHS = 10

optimizer = SGD(best_model.parameters(), lr=2e-4)

train_loader = dataloaders['tune']["train"]
valid_loader = dataloaders['tune']["val"]

train(search_space = best_model,
      train_loader = train_loader,
      valid_loader = valid_loader,
      optimizer = optimizer,
      enot_optimizer = None,
      n_epochs = N_EPOCHS,
      n_warmup_epochs = 0,
      phase = "tune")